# Imports

In [1]:
from urllib.parse import urlparse, parse_qs, urlencode
from requests import get, post, Session
from getpass import getpass
from re import findall

# Get token

## Urls def

In [2]:
beatport_base_url = 'https://api.beatport.com/v4'
beatport_redirect_uri = beatport_base_url + '/auth/o/post-message/'

## Get client id

In [3]:
client_id = ''
html = get('https://api.beatport.com/v4/docs/').content.decode('utf-8')
scripts_matches = findall(r"src=.(.*js)", html)
for script_url in scripts_matches:
    print(script_url)
    url = 'https://api.beatport.com{}'.format(script_url)
    js = get(url.format(script_url)).content.decode('utf-8')
    client_id_matches = findall(r"API_CLIENT_ID: \'(.*)\'", js)
    if client_id_matches:
        client_id = client_id_matches[0]
client_id

/static/btprt/sentry.js
/static/btprt/cb7c25830019c7384e99.js
/static/btprt/5a6a7454820aa6fc6366.js
/static/btprt/b08ba1200741e40ed1e7.js
/static/btprt/dfbe36b305cf113177e7.js
/static/btprt/6a17459eef4fe0bc0b50.js
/static/btprt/7aff86dea244ea63662a.js


'0GIvkCltVIuPkkwSJHp6NDb3s0potTjLBQr388Dd'

In [4]:
client_id = '0GIvkCltVIuPkkwSJHp6NDb3s0potTjLBQr388Dd'

## Login

In [5]:
sess = Session()

In [14]:
response = sess.post(url=beatport_base_url + '/auth/login', 
                     json={'username': input("Enter your username: "),
                           'password': getpass("Enter your password: ")})

KeyboardInterrupt: 

In [ ]:
response

<Response [200]>

## Authorization code

In [8]:
query = urlencode({
    'client_id': client_id,
    'response_type': 'code',
    'redirect_uri': beatport_redirect_uri
})
response = sess.get(url=f'{beatport_base_url}/auth/o/authorize/?{query}', allow_redirects=False)
response.url

'https://api.beatport.com/v4/auth/o/authorize/?client_id=0GIvkCltVIuPkkwSJHp6NDb3s0potTjLBQr388Dd&response_type=code&redirect_uri=https%3A%2F%2Fapi.beatport.com%2Fv4%2Fauth%2Fo%2Fpost-message%2F'

In [9]:
next_url = urlparse(beatport_base_url + response.headers['Location'])
auth_code = parse_qs(next_url.query)['code'][0]
auth_code

'Qcj5Jx4QGkuE7LkJvZ8zxmuwd90xeg'

In [10]:
parse_qs(next_url.query)

{'code': ['Qcj5Jx4QGkuE7LkJvZ8zxmuwd90xeg'],
 'target': ['https://api.beatport.com']}

In [11]:
query = urlencode({
    'code': auth_code,
    'grant_type': 'authorization_code',
    'redirect_uri': beatport_redirect_uri,
    'client_id': client_id
})
response = sess.post(url=beatport_base_url+'/auth/o/token/?'+query)
token = response.json()
token

{'access_token': 'y4tfISMw6VvowM8y6MKQ5WFnHS5oMn',
 'expires_in': 36000,
 'token_type': 'Bearer',
 'scope': 'app:docs user:dj',
 'refresh_token': 'Ro90w6ChJdzUL1dVPQCKyrluNq3v5N'}

## Test requests

In [12]:
headers = {'User-Agent': 'Mozilla/5.0 (X11; Ubuntu; Linux x86_64; rv:81.0) Gecko/20100101 Firefox/81.0',
           'Authorization': f'Bearer {token["access_token"]}'}

In [13]:
response = get(url=beatport_base_url+'/catalog/search/', 
               headers=headers, 
               params={"q": "lay low tiesto", 'type': 'tracks', 'per_page': '5'})
response.json()

{'tracks': [{'artists': [{'id': 4178,
     'image': {'id': 26803617,
      'uri': 'https://geo-media.beatport.com/image_size/590x404/846683c6-4b4d-44db-8ebe-13d5d3cd5f92.jpg',
      'dynamic_uri': 'https://geo-media.beatport.com/image_size/{w}x{h}/846683c6-4b4d-44db-8ebe-13d5d3cd5f92.jpg'},
     'name': 'Tiesto',
     'slug': 'tiesto',
     'url': 'https://api.beatport.com/v4/catalog/artists/4178/'}],
   'publish_status': 'published',
   'available_worldwide': False,
   'bpm': 122,
   'catalog_number': '5054197482496',
   'current_status': {'id': 10,
    'name': 'General Content',
    'url': 'https://api.beatport.com/v4/auxiliary/current-status/10/'},
   'encoded_date': '2022-12-02T13:08:56-07:00',
   'exclusive': False,
   'free_downloads': [],
   'free_download_start_date': None,
   'free_download_end_date': None,
   'genre': {'id': 39,
    'name': 'Dance / Electro Pop',
    'slug': 'dance-electro-pop',
    'url': 'https://api.beatport.com/v4/catalog/genres/39/'},
   'id': 17189283,
